<img src="plcparser_icon.png" />

<center>$A \land B \land C$</center>

# PLCParser implemented in [Hy](https://github.com/hylang/hy) ~language

See also:

- Propositional Logic Clause Parser [main project hub](https://github.com/markomanninen/PLCParser)
- Jupyter notebook with Python [kernel](http://nbviewer.jupyter.org/github/markomanninen/PLCParser/blob/master/Hy%20-level%20PLCParser.ipynb)


## Import library

In [1]:
(require [hyPLCParser.plcparser [*]])
(import (hyPLCParser.plcparser (*)))

Show available operators

In [2]:
(print operators)

['is_nope', '¬', 'is_and', '∧', 'is_nand', '↑', 'is_or', '∨', 'is_nor', '↓', 'is_xor', '⊕', 'is_xnor', '↔']


Evaluate clause

In [3]:
#$(1 and? (and? 1))

True

In [4]:
#$(and? 1 "⊤" True)

True

## Tests

In [5]:
(assert (= #$(or? 0) False))
(assert (= #$(and? 0) False))
(assert (= #$(xor? 0) False))
(assert (= #$(nope? 1) False))
(assert (= #$(nand? 1) False))
(assert (= #$(nor? 1) False))
(assert (= #$(xnor? 1) False))
(assert (= #$(1) True))
(assert (= #$(0) False))
(assert (= #$(True) True))
(assert (= #$(False) False))
(assert (= #$1 True))
(assert (= #$0 False))
(assert (= #$True True))
(assert (= #$False False))
(assert (= #$() False))
(assert (= #$("") ""))
(assert (= #$(None) None))
(assert (= #$(¬ 0 False ⊥) True))

(assert (= #$(1 and? 1) True))
(assert (= #$(1 or? 0) True))
(assert (= #$(1 xor? 0) True))
(assert (= #$(1 nand? 1) False))
(assert (= #$(1 nor? 0) False))
(assert (= #$(1 xnor? 0) False))
(assert (= #$(¬ ⊤) False))
(assert (= #$( ⊥ ∨ ⊤ ) True))
(assert (= #$(1 xnor? 0) False))
(assert (= #$(⊕ 1 0) True))
(assert (= #$( (1 and? 0) and? (1 and? 1) ) False))
(assert (= #$(and? 1 1 (1 and? (and? 1 (1 nand? 1 nand? 1)))) True))

(assert (= #$(and?) None)) ; expression error

Expression error!


## Code

In [6]:
;(eval-when-compile (setv operators []))
(eval-and-compile (setv operators []))

; define math operands
(setv ⊤ 1)
(setv ⊥ 0)

; add operators to global variable so that on parser loop
; we can use it on if clauses
(defreader > [item]
  (if-not (in item operators)
    (.append operators item)))

; define operator function and math alias
; plus set them to operators global list
(defmacro defoperator [op-name op-symbol params &rest body]
  `(do 
    (defn ~op-name ~params ~@body)
    (setv ~op-symbol ~op-name)
    #>~op-name
    #>~op-symbol))

; define true comparison function
(defn true? [value] 
  (or (= value 1) (= value True) (= value "True")  (= value "⊤")))

; same as nor at the moment...
(defoperator nope? ¬ [&rest truth-list] 
  (not (any truth-list)))
  ;(not (any (map true? truth-list))))

; and operation : zero or more arguments, zero will return false, 
; otherwise all items needs to be true
(defoperator and? ∧ [&rest truth-list]
  (all (map true? truth-list)))

; negation of and
(defoperator nand? ↑ [&rest truth-list]
  (not (apply and? truth-list)))

; or operation : zero or more arguments, zero will return false, 
; otherwise at least one of the values needs to be true
(defoperator or? ∨ [&rest truth-list]
  (any (map true? truth-list)))

; negation of or
(defoperator nor? ↓ [&rest truth-list]
  (not (apply or? truth-list)))

; xor operation (parity check) : zero or more arguments, zero will return false, 
; otherwise odd number of true's is true
(defoperator xor? ⊕ [&rest truth-list]
    (setv boolean False)
    (for [truth-value truth-list]
        (if (true? truth-value)
            (setv boolean (not boolean))))
    boolean)

; negation of xor
(defoperator xnor? ↔ [&rest truth-list]
  (not (apply xor? truth-list)))

; main parser loop for propositional logic clauses
; todo: operator precedence!
(defreader $ [code]
  (if
    ; if scalar value, return that
    (not (coll? code)) code
    ; else if empty list, return false
    (= (len code) 0) False
    ; else if list with lenght of 1 and that single item is not the operator
    (and (= (len code) 1) (not (in (get code 0) operators)))
      `#$~@code
    ; else if list with three items, and the operator is in the middle (infix)
    (and (= (len code) 3) (in (get code 1) operators))
      `(~(get code 1) #$~(get code 0) #$~(get code 2))
    ; else if list with two or more items, and the second is the operator
    (and (> (len code) 2) (in (get code 1) operators))
      (do
        ; take first two items and reverse
        (setv a (doto (list (take 2 code)) (.reverse)))
        ; take rest of the items after second item
        (setv b (list (drop 2 code)))
        ; b could be empty
        (if (> (len b) 0)
          `(~@a #$~b)
          `(~@a)))
    ; else if list with more items than 1 and the first item the operator
    (and (> (len code) 1) (in (get code 0) operators))
      (do
        ; take the first item i.e. operator
        (setv a (list (take 1 code)))
        ; append all numeric items after operator to a and flatten list
        (.append a (list (take-while 
          ; support numeric operands for math equations
          (fn [x] (or (numeric? x) (in x ["1" "True" "⊤" "0" "False" "⊥"]))) (drop 1 code))))
        (setv a (flatten a))
        ; after the first items seek non numeric i.e. list
        (setv b (list (drop-while 
          ; support numeric operands for math equations
          (fn [x] (or (numeric? x) (in x ["1" "True" "⊤" "0" "False" "⊥"]))) (drop 1 code))))
        ; b could be empty
        (if (> (len b) 0)
          `(~@a #$(~@b))
          `(~@a)))
    ; else possibly syntax error on clause
    `(print "Expression error!")))

## The [MIT](https://choosealicense.com/licenses/mit/) License

Copyright © 2017 Marko Manninen